In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
from functools import reduce

data = pd.read_pickle("../data/df_preproc.pk")
hashtags_vocab = reduce((lambda x, y: set(x).union(y)), data.Hashtags)
mentions_vocab = reduce((lambda x, y: set(x).union(y)), data.Mentions)

<IPython.core.display.Javascript object>

In [3]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras.layers import Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


def embedding_model(vocab, max_len, max_features, out_dim):
    vectorize_layer = TextVectorization(
        max_tokens=max_features,
        output_sequence_length=max_len,
        output_mode="int",
        # split=None,
        vocabulary=list(vocab),
    )
    return tf.keras.Sequential(
        layers=[vectorize_layer, Embedding(max_features + 1, out_dim), Flatten()]
    )


max_hashtags = 5
max_mentions = 5

_embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/2")
_hashtag_embed = embedding_model(hashtags_vocab, 8, 2000, 16)
_mention_embed = embedding_model(mentions_vocab, 8, 1500, 16)

<IPython.core.display.Javascript object>

In [4]:
print(_hashtag_embed.predict([["google apple"], ["app quote"]]).shape)
print(_mention_embed.predict([["google apple"], ["tesla"]]).shape)
print(_embed(data.Tweet[10:15]).shape)

(2, 128)
(2, 128)
(5, 128)


<IPython.core.display.Javascript object>

In [6]:
from thinc.api import (
    Model,
    chain,
    strings2arrays,
    with_array,
    HashEmbed,
    expand_window,
    Relu,
    Softmax,
    Adam,
    warmup_linear,
    TensorFlowWrapper,
)
from thinc.types import List1d, Array2d

embed_layer = hub.KerasLayer(
    "https://tfhub.dev/google/nnlm-en-dim128/2", input_shape=[], dtype=tf.string
)
embed = tf.keras.Sequential()
embed.add(embed_layer)
embed: Model[List1d, Array2d] = TensorFlowWrapper(embed)

<IPython.core.display.Javascript object>

In [ ]:
n_hidden = 64
dropout = 0.2

with Model.define_operators({">>": chain}):
    model = (
        embed
        >> Relu(nO=n_hidden, dropout=dropout)
        >> Relu(nO=n_hidden, dropout=dropout)
        >> Softmax()
    )